In [28]:
#STREAMS2TABLE
#written by Zachary J.J. Roper
#z.roper@vanderbilt.edu, zach@roper.io

#A script for converting raw event codes and timestamps into a trial-item dataset

In [29]:
#This notebook requires the following modules
import os
import numpy
import csv

In [30]:
#READ IN EVENT CODES
#The prepared file, out.csv, is a 2 column x session length (based on total number of trials) consistinting of event 
#codes (EVENTS[]) and their paired timestamps(TIMES[]). The file streams2text.m was used to generate these data 
#from the TDT data dump.

In [31]:
with open('out.csv') as csvfile:
    data = csv.reader(csvfile, delimiter = ',')
    events = []
    times = []
    for row in data:
        event = row[0]
        events.append(event)
        time = row[1]
        times.append(time)
pairedlist = list(zip(events,times))

In [32]:
events.count('2756') #catch trials

3

In [33]:
trial = 0
triallist = [None]*len(pairedlist)
eclist = [None]*len(pairedlist)
for i, data in enumerate(pairedlist):
    if pairedlist[i][0] == '2301':
        trial += 1
    triallist[i] = trial  
    eclist[i] = i
uberlist = list(zip(triallist,eclist,pairedlist))

In [34]:
#uberlist

In [35]:
'2810' in uberlist[74][2]

False

In [36]:
RT = [None]*len(pairedlist)
RTbeg = 0*len(pairedlist)
RTend = 0*len(pairedlist)
for i, trial in enumerate(uberlist):
    #print(i,trial)
    if '2651' in uberlist[i][2]:
        RTbeg = float(uberlist[i][2][1])
        focusTrial = uberlist[i][0]
        for p, case in enumerate(uberlist):
            if '2810' in uberlist[p][2] and uberlist[i][2] == focusTrial:
                RTend = float(uberlist[p][2][1])
    RT[i] = RTend - RTbeg

In [37]:
#RT

In [38]:
#The following provides indexed lists for each of the individual event codes of interest 

In [39]:
TargetOnset = []
FixOnset = []
Saccade = []
for i, x in enumerate(events):
    #print(i,x)
    if x == '2301':
        FixOnset.append(i)
    if x == '2651':
        TargetOnset.append(i)
    if x == '2810':
        Saccade.append(i)

In [40]:
#float(times[74])-float(times[70])

In [41]:
#Saccade

In [42]:
#for i, x in enumerate(Saccade):
#    print(Saccade[i],TargetOnset[i])

In [43]:
TOtimes = []
for items in TargetOnset:
    TOtimes.append(times[items])

In [44]:
Trials = []
TimeTrials = []
for i, x in enumerate(FixOnset):
    if i < len(FixOnset)-1:
        a = FixOnset[i]
        b = FixOnset[i+1]-1
        Trials.append(events[a:b])
        TimeTrials.append(times[a:b])
    else:
        a = FixOnset[i]
        Trials.append(events[a:])
        TimeTrials.append(times[a:])

In [45]:
#print(TimeTrials[0])

In [46]:
path = os.getcwd()
csvfile = (path+"/table.csv")
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(Trials)

In [47]:
accuracy = [None]*len(Trials)
abort = [None]*len(Trials)
error = [None]*len(Trials)
catch = [None]*len(Trials)
correct = [None]*len(Trials)
tloc = [None]*len(Trials)
for i, trial in enumerate(Trials):
    if '2600' in trial:
        accuracy[i] = 1 #Correct Saccade (2600)
    else:
        accuracy[i] = 0  
    if '2651' in trial: #fixation was maintained sufficiently long enough for target onset (2651)
        abort[i] = 0
    else:
        abort[i] = 1
    if '887' in trial:
        error[i] = 1
    else:
        error[i] = 0
    if '2756' in trial:
        catch[i] = 1
    else:
        catch[i] = 0
    if '888' in trial:
        correct[i] = 1
    else:
        correct[i] = 0
    #if '5000' in trial:
    #    tloc[i] = 360
    if '5045' in trial:
        tloc[i] = 45
    elif '5090' in trial:
        tloc[i] = 90
    elif '5135' in trial:
        tloc[i] = 135
    elif '5180' in trial:
        tloc[i] = 180
    elif '5225' in trial:
        tloc[i] = 225
    elif '5270' in trial:
        tloc[i] = 270
    elif '5315' in trial:
        tloc[i] = 315
    else:
        tloc[i] = 0

In [48]:
new_dict = {}
for i, trial in enumerate(Trials):
    new_dict[i+1] = dict(zip(Trials[i],TimeTrials[i]))

In [49]:
RT = [0]*len(Trials)
focusTrial = 0
for i, trial in enumerate(Trials):
    if '2651' in trial and '2810' in trial: #and '2600' in trial:
        focusTrial = i + 1
        for p, case in enumerate(uberlist):
            if uberlist[p][0] == focusTrial:
                if '2810' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTend = float(uberlist[p][2][1])
                if '2651' in uberlist[p][2][0]:
                    #print(uberlist[p][2][1])
                    RTbeg = float(uberlist[p][2][1])
            RT[i] = RTend - RTbeg

In [50]:
#RT

In [51]:
checksum = sum(error)+sum(abort)+sum(catch)

In [52]:
checksum

588

In [53]:
path = os.getcwd()
csvfile = (path+"/behaviorArray.csv")
trial = 0
triallist = [None]*len(abort)
with open(csvfile, "w") as output:
    for i,x in enumerate(abort):
        trial += 1
        triallist[i] = trial 
        print(i+1,abort[i],accuracy[i],error[i],correct[i],catch[i],RT[i],tloc[i])
    header = ['Trial', 'Abort', 'Accuracy', 'Error', 'Correct', 'Catch', 'RT', 'TarLoc']    
    dataset = list(zip(triallist,abort,accuracy,error,correct,catch,RT,tloc))
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(dataset)

1 1 0 0 0 0 0 270
2 0 0 1 0 0 181.98509999999987 0
3 0 0 0 1 0 166.0108999999993 90
4 1 0 0 0 0 0 180
5 0 0 1 0 0 182.1081000000013 180
6 1 0 0 0 0 0 45
7 0 0 0 1 0 221.96220000000176 135
8 1 0 0 0 0 0 225
9 0 1 0 1 0 212.00890000000072 45
10 0 1 0 1 0 208.03600000000006 0
11 0 1 0 1 0 177.9713999999949 225
12 1 0 0 0 0 0 45
13 0 1 0 1 0 169.9838000000018 270
14 0 0 1 0 0 144.0155999999988 90
15 1 0 0 0 0 0 225
16 0 0 1 0 0 173.99789999999484 45
17 0 1 0 1 0 197.95960000000196 45
18 1 0 0 0 0 0 270
19 1 0 0 0 0 0 45
20 0 1 0 1 0 164.00389999999607 135
21 1 0 0 0 0 0 180
22 0 1 0 1 0 178.0121000000072 225
23 0 1 0 1 0 219.9551999999967 45
24 0 1 0 1 0 200.00770000000193 225
25 0 1 0 1 0 157.98279999999795 90
26 1 0 0 0 0 0 0
27 0 1 0 1 0 166.01080000000366 180
28 1 0 0 0 0 0 135
29 1 0 0 0 0 0 0
30 0 1 0 1 0 181.98539999999048 45
31 0 1 0 1 0 167.97709999998915 270
32 0 1 0 1 0 167.9771000000037 45
33 1 0 0 0 0 0 225
34 1 0 0 0 0 0 270
35 1 0 0 0 0 0 135
36 1 0 0 0 0 0 180
37 0 1 0 1 0 

487 1 0 0 0 0 0 270
488 0 0 1 0 0 271.9743000001181 270
489 0 0 1 0 0 301.9981000002008 225
490 1 0 0 0 0 0 135
491 1 0 0 0 0 0 135
492 1 0 0 0 0 0 135
493 1 0 0 0 0 0 315
494 0 1 0 1 0 205.98790000006557 135
495 1 0 0 0 0 0 315
496 0 1 0 1 0 219.99620000016876 135
497 0 1 0 1 0 231.99749999982305 0
498 1 0 0 0 0 0 225
499 1 0 0 0 0 0 45
500 0 1 0 1 0 215.9819000000134 135
501 1 0 0 0 0 0 0
502 0 0 0 1 0 188.00640000007115 45
503 0 1 0 1 0 199.96659999992698 270
504 0 0 1 0 0 247.971700000111 135
505 0 0 0 1 0 40.01809999998659 315
506 1 0 0 0 0 0 90
507 0 1 0 1 0 210.00199999986216 45
508 0 1 0 1 0 213.97519999998622 270
509 1 0 0 0 0 0 225
510 0 1 0 1 0 212.00889999978244 90
511 1 0 0 0 0 0 135
512 1 0 0 0 0 0 315
513 0 1 0 1 0 205.98790000006557 135
514 1 0 0 0 0 0 135
515 0 1 0 1 0 199.96669999998994 315
516 1 0 0 0 0 0 90
517 1 0 0 0 0 0 270
518 0 0 1 0 0 211.96790000004694 45
519 0 1 0 1 0 233.96350000007078 315
520 1 0 0 0 0 0 0
521 1 0 0 0 0 0 180
522 0 1 0 1 0 193.986399999819

797 0 1 0 1 0 187.96539999987 45
798 0 0 0 1 0 67.95259999996051 135
799 0 1 0 1 0 204.02169999992475 180
800 1 0 0 0 0 0 0
801 0 1 0 1 0 198.04160000011325 180
802 0 1 0 1 0 195.9936999999918 315
803 1 0 0 0 0 0 180
804 1 0 0 0 0 0 135
805 0 1 0 1 0 195.99359999992885 45
806 0 1 0 1 0 205.98790000006557 315
807 1 0 0 0 0 0 90
808 0 1 0 1 0 182.02619999996386 0
809 1 0 0 0 0 0 225
810 1 0 0 0 0 0 270
811 0 1 0 1 0 213.97519999998622 0
812 0 1 0 1 0 169.98379999981262 135
813 0 1 0 1 0 198.00059999991208 225
814 1 0 0 0 0 0 180
815 1 0 0 0 0 0 135
816 0 1 0 1 0 188.00640000007115 45
817 1 0 0 0 0 0 315
818 0 0 1 0 0 235.9706999999471 90
819 1 0 0 0 0 0 135
820 1 0 0 0 0 0 270
821 0 1 0 1 0 195.9936999999918 225
822 0 1 0 1 0 234.00450000003912 90
823 1 0 0 0 0 0 270
824 0 0 1 0 0 234.00450000003912 0
825 0 1 0 1 0 205.98789999983273 0
826 1 0 0 0 0 0 180
827 0 1 0 1 0 234.00450000003912 270
828 1 0 0 0 0 0 45
829 0 1 0 1 0 203.98090000008233 0
830 1 0 0 0 0 0 270
831 0 0 1 0 0 240.02560

1127 0 1 0 1 0 229.99050000030547 135
1128 0 1 0 1 0 223.96950000012293 0
1129 0 1 0 1 0 193.98650000011548 135
1130 0 1 0 1 0 198.04169999994338 315
1131 1 0 0 0 0 0 225
1132 0 1 0 1 0 231.99750000005588 180
1133 1 0 0 0 0 0 315
1134 0 1 0 1 0 196.03439999977127 315
1135 0 1 0 1 0 189.97259999997914 180
1136 0 1 0 1 0 217.98919999971986 0
1137 0 1 0 1 0 176.00510000018403 135
1138 1 0 0 0 0 0 0
1139 0 1 0 1 0 213.97500000009313 45
1140 0 1 0 1 0 166.01080000028014 45
1141 0 1 0 1 0 187.96559999976307 180
1142 1 0 0 0 0 0 225
1143 1 0 0 0 0 0 0
1144 1 0 0 0 0 0 180
1145 0 1 0 1 0 195.9936999999918 45
1146 0 1 0 1 0 198.00070000020787 225
1147 1 0 0 0 0 0 90
1148 0 1 0 1 0 173.99789999984205 45
1149 1 0 0 0 0 0 225
1150 0 1 0 1 0 211.96790000004694 0
1151 1 0 0 0 0 0 225
1152 0 1 0 1 0 205.9879999998957 225
1153 0 1 0 1 0 229.99019999988377 225
1154 1 0 0 0 0 0 90
1155 0 1 0 1 0 177.97110000019893 225
1156 1 0 0 0 0 0 45
1157 1 0 0 0 0 0 315
1158 1 0 0 0 0 0 135
1159 1 0 0 0 0 0 45
1160

In [54]:
sum(catch)

3